# Create CNN for CIFAR10 dataset

1. read Toronto paper.
2. Make a subroutine to get the predictions of the top 3 probabilities for each test sample and evaluation the overall accuracy of top1 and top3 predictions.
3. read CNN overview pdf
4. download cifar10 dataset.
5. make your first CNN.  It does not need to be a big one.  It is OK to get just one or two convolution layers in your first CNN to save time.
